<a href="https://colab.research.google.com/github/carloscastillo10/predicting-academic-performance/blob/main/annex-f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CASO 4

En este caso se toma como referencia https://www.kaggle.com/code/devassaxd/student-performance-prediction-complete-analysis/notebook#Part-3:-Modeling

Se tuvieron en cuenta los siguientes aspectos:
* Se escaló a 10 puntos cada una de las notas.
* Se obtuvo la nota ponderada (columna: *`ponderado`*) de cada componente.
    * Por ejemplo si la nota del componente **`ACDB1`**  es de **`8.2`** y el peso de este componente es del **`35%`** y con una nota máxima de **`10`**, entonces se hace la regla de tres: `si la nota maxima (10) equivale al 100% a cuanto equivale la nota obtenida por un estudiante en ese componente (ACDB1)`
* Se obtuvo la **`tasa de aprobacion`** total del componente, esta columna se calcula con la siguiente fórmula: `total de componentes aprobados` / `total de componentes totales de la materia`, para definir cuando se aprueba o reprueba un componente se toma como referencia la siguiente tabla:

>> | componente | Descripcion | peso | nota máxima | nota mínima para aprobar |
| ---------- | ---- | ---- | ----------- | ------------------------ |
| ACDB1      | Aprendizaje en contacto con el docente primer bimestre | 35%  | 10         | 7                     |
| APEB1      | Aprendizaje práctico experimental primer bimestre | 30%  | 10           | 7                     |
| AAB1       | Aprendizaje autónomo primer bimestre | 35%  | 10         | 7                     |
| ACDB2      | Aprendizaje en contacto con el docente segundo bimestre | 35%  | 10        | 7                     |
| APEB2      | Aprendizaje práctico experimental segundo bimestre | 30%  | 10           | 7                     |
| AAB2       | Aprendizaje autónomo segundo bimestre | 35%  | 10         | 7                    |
| FINAL      | Nota final del bimestre | 100% | 10          | 7                        |

In [ ]:
import pandas as pd
pd.options.display.max_columns = 50

## Leer datos de estudiantes, actividades y notas

In [ ]:
student_info_data = pd.read_csv('data/estudentInfo.csv')
assessments_data = pd.read_csv('data/assessments.csv')
student_assessments_data = pd.read_csv('data/studentAssessment.csv')

## Actividades

In [ ]:
final_scores = assessments_data[assessments_data.componente == 'FINAL']
component_scores = assessments_data[assessments_data.componente != 'FINAL']
amounts = component_scores.groupby(
    ['curso_nombre', 'codigo_periodo']
).count()['componente'].reset_index()

amounts[
    (amounts.curso_nombre == 'COMUNICACION DE DATOS')
    & (amounts.codigo_periodo == 202062)
]

,curso_nombre,codigo_periodo,componente
74,COMUNICACION DE DATOS,202062,6


In [ ]:
component_approval_notes = {
    'ACDB1': {'original_max_note': 3.5, 'max_note': 10, 'aproval_note': 7}, 
    'APEB1': {'original_max_note': 3, 'max_note': 10, 'aproval_note': 7},
    'AAB1': {'original_max_note': 3.5, 'max_note': 10, 'aproval_note': 7},
    'ACDB2': {'original_max_note': 3.5, 'max_note': 10, 'aproval_note': 7}, 
    'APEB2': {'original_max_note': 3, 'max_note': 10, 'aproval_note': 7},
    'AAB2': {'original_max_note': 3.5,  'max_note': 10, 'aproval_note': 7},
    'FINAL': {'original_max_note': 10, 'max_note': 10, 'aproval_note': 7}
}

def scaled_note(component, score):
    if component != 'FINAL':
        max_note = component_approval_notes[component]['max_note']
        original_max_note = component_approval_notes[component]['original_max_note']
        
        return round(((score * max_note) / original_max_note), 2)
    return score

def set_pass_fail(component, score):
    if score >= component_approval_notes[component]['aproval_note']:
        return True
    
    return False

def set_weighted_grade(component, score, weigth):
    max_note_component = component_approval_notes[component]['max_note']
    
    return round(((score * weigth) / max_note_component), 2)

student_assessments = pd.merge(
    student_assessments_data, 
    assessments_data,
    how = 'inner',
    on = ['curso_nombre', 'codigo_periodo', 'componente']
)

student_assessments['nota'] = student_assessments.apply(
    lambda row: scaled_note(row.componente, row.nota), 
    axis = 1
)

student_assessments['aprobado'] = student_assessments.apply(
    lambda row: set_pass_fail(row.componente, row.nota), 
    axis = 1
)

student_assessments['ponderado'] = student_assessments.apply(
    lambda row: set_weighted_grade(row.componente, row.nota, row.peso),
    axis = 1
)

student_assessments[
    (student_assessments.fake_identificacion == 'SLXA73875862178452')
    & (student_assessments.curso_nombre == 'COMUNICACION DE DATOS')
    & (student_assessments.codigo_periodo == 202062)
]

,fake_identificacion,codigo_periodo,curso_nombre,componente,nota,peso,aprobado,ponderado
0,SLXA73875862178452,202062,COMUNICACION DE DATOS,AAB1,5.57,35,False,19.50
26210,SLXA73875862178452,202062,COMUNICACION DE DATOS,ACDB1,9.51,35,True,33.28
52420,SLXA73875862178452,202062,COMUNICACION DE DATOS,APEB1,1.70,30,False,5.10
78630,SLXA73875862178452,202062,COMUNICACION DE DATOS,AAB2,5.74,35,False,20.09
104840,SLXA73875862178452,202062,COMUNICACION DE DATOS,ACDB2,3.86,35,False,13.51
131050,SLXA73875862178452,202062,COMUNICACION DE DATOS,APEB2,5.17,30,False,15.51
157260,SLXA73875862178452,202062,COMUNICACION DE DATOS,FINAL,5.30,100,False,53.00


In [ ]:
scores_data = pd.pivot(
    student_assessments,
    index = ['fake_identificacion', 'codigo_periodo', 'curso_nombre'],
    values = 'nota',
    columns = 'componente'
)
scores_data.columns = [f'{column_name.lower()}_nota' for column_name in scores_data.columns]
scores_data = scores_data.reset_index()

weigtheds_grade_data = pd.pivot(
    student_assessments,
    index = ['fake_identificacion', 'codigo_periodo', 'curso_nombre'],
    values = 'ponderado',
    columns = 'componente'
)
weigtheds_grade_data.columns = [f'{column_name.lower()}_ponderado' for column_name in weigtheds_grade_data.columns]
weigtheds_grade_data = weigtheds_grade_data.reset_index()

student_scores_weigtheds = pd.merge(
    scores_data,
    weigtheds_grade_data,
    how = 'inner',
    on = ['fake_identificacion', 'codigo_periodo', 'curso_nombre']
)

student_scores_weigtheds[
    (student_scores_weigtheds.fake_identificacion == 'SLXA73875862178452')
    & (student_scores_weigtheds.curso_nombre == 'COMUNICACION DE DATOS')
    & (student_scores_weigtheds.codigo_periodo == 202062)
]

,fake_identificacion,codigo_periodo,curso_nombre,aab1_nota,aab2_nota,acdb1_nota,acdb2_nota,apeb1_nota,apeb2_nota,final_nota,aab1_ponderado,aab2_ponderado,acdb1_ponderado,acdb2_ponderado,apeb1_ponderado,apeb2_ponderado,final_ponderado
18255,SLXA73875862178452,202062,COMUNICACION DE DATOS,5.57,5.74,9.51,3.86,1.7,5.17,5.3,19.5,20.09,33.28,13.51,5.1,15.51,53.0


In [ ]:
student_pass_rate = pd.merge(
    student_assessments[
        (student_assessments.aprobado == True)
        & (student_assessments.componente != 'FINAL')
    ].groupby(
        ['fake_identificacion', 'codigo_periodo', 'curso_nombre']
    ).count()['aprobado'].reset_index(),
    amounts,
    how = 'left',
    on = ['curso_nombre', 'codigo_periodo']
)

student_pass_rate['tasa_aprobacion'] = round((student_pass_rate['aprobado'] / student_pass_rate['componente']), 2)
student_pass_rate.drop(['aprobado', 'componente'], axis = 1, inplace = True)

student_pass_rate[
    (student_pass_rate.fake_identificacion == 'SLXA73875862178452')
    & (student_pass_rate.curso_nombre == 'COMUNICACION DE DATOS')
    & (student_pass_rate.codigo_periodo == 202062)
]

,fake_identificacion,codigo_periodo,curso_nombre,tasa_aprobacion
15906,SLXA73875862178452,202062,COMUNICACION DE DATOS,0.17


# Integración de datos

In [ ]:
assessment_info = pd.merge(
    student_scores_weigtheds,
    student_pass_rate,
    how = 'inner',
    on = ['fake_identificacion', 'codigo_periodo', 'curso_nombre']
)

assessment_info[
    (assessment_info.fake_identificacion == 'SLXA73875862178452')
    & (assessment_info.curso_nombre == 'COMUNICACION DE DATOS')
    & (assessment_info.codigo_periodo == 202062)
]

,fake_identificacion,codigo_periodo,curso_nombre,aab1_nota,aab2_nota,acdb1_nota,acdb2_nota,apeb1_nota,apeb2_nota,final_nota,aab1_ponderado,aab2_ponderado,acdb1_ponderado,acdb2_ponderado,apeb1_ponderado,apeb2_ponderado,final_ponderado,tasa_aprobacion
15906,SLXA73875862178452,202062,COMUNICACION DE DATOS,5.57,5.74,9.51,3.86,1.7,5.17,5.3,19.5,20.09,33.28,13.51,5.1,15.51,53.0,0.17


In [ ]:
student_info_data[
    (student_info_data.fake_identificacion == 'SLXA73875862178452')
    & (student_info_data.curso_nombre == 'COMUNICACION DE DATOS')
    & (student_info_data.codigo_periodo == 202062)
]

,fake_identificacion,ciclo_academico,codigo_programa,codigo_malla,codigo_periodo,region,zona,centro,tipo_de_est_programa,tipo_de_ingreso,periodo_admision,estatus_cursos,estatus_de_pago,descripcion_de_becas,porcentaje_de_beca,descripcion_descuento,porcentaje_des_matricula,monto_descuento_automatico,convenios_y_contratos,formas_de_pago,tipo_de_pago,provincia,canton,edad,sexo,discapacidad,tipo_discapacidad,porcentaje_de_discapacidades,numero_discapacidades,curso_codigo,curso_nombre,numero_reprobaciones,status
22310,SLXA73875862178452,3,INFO_D1,IFD6,202062,GUAYAQUIL,GRAN-GYE,GUAYAQUIL,CONTINUO,ADAPTACION DE CARRERA/MALLA,202062.0,CURSOS INSCRITOS,CON PAGO TOTAL,NaN,NaN,DESCUENTO POR NUMERO DE COMPONENTES GRADO DIST...,5.0,80.23,NaN,TARJETA ONLINE,PAGO CON TARJETA,NO DEFINIDO,NO DEFINIDO,32.0,HOMBRE,NO,NO DEFINIDO,0,0,DRBD_3019,COMUNICACION DE DATOS,2,REPROBADA


In [ ]:
student_info_columns = list(student_info_data.columns)
assessment_columns = [
    'aab1_nota', 'aab1_ponderado', 
    'acdb1_nota',  'acdb1_ponderado', 
    'apeb1_nota', 'apeb1_ponderado', 
    'aab2_nota', 'aab2_ponderado', 
    'acdb2_nota', 'acdb2_ponderado', 
    'apeb2_nota', 'apeb2_ponderado',
    'final_nota', 'final_ponderado',
    'tasa_aprobacion'
]
selected_columns = student_info_columns + assessment_columns

In [ ]:
integrated__data_case4 = pd.merge(
    student_info_data,
    assessment_info,
    how = 'inner',
    on = ['fake_identificacion', 'codigo_periodo', 'curso_nombre']
)[selected_columns]

integrated__data_case4[
    (integrated__data_case4.fake_identificacion == 'XTLA48155273548078')
    & (integrated__data_case4.curso_nombre == 'INGENIERIA DE REQUISITOS')
    & (integrated__data_case4.codigo_periodo == 202164)
]

,fake_identificacion,ciclo_academico,codigo_programa,codigo_malla,codigo_periodo,region,zona,centro,tipo_de_est_programa,tipo_de_ingreso,periodo_admision,estatus_cursos,estatus_de_pago,descripcion_de_becas,porcentaje_de_beca,descripcion_descuento,porcentaje_des_matricula,monto_descuento_automatico,convenios_y_contratos,formas_de_pago,tipo_de_pago,provincia,canton,edad,sexo,discapacidad,tipo_discapacidad,porcentaje_de_discapacidades,numero_discapacidades,curso_codigo,curso_nombre,numero_reprobaciones,status,aab1_nota,aab1_ponderado,acdb1_nota,acdb1_ponderado,apeb1_nota,apeb1_ponderado,aab2_nota,aab2_ponderado,acdb2_nota,acdb2_ponderado,apeb2_nota,apeb2_ponderado,final_nota,final_ponderado,tasa_aprobacion
4,XTLA48155273548078,6,INFO_D1,IFD6,202164,QUITO,METROPOLIT,QUITO-TURUBAMBA,CONTINUO,ADAPTACION DE CARRERA/MALLA,202062.0,CURSOS INSCRITOS,CON PAGO TOTAL,CONVENIO DE ADJUDICACIÓN DE BE,10.0,DESCUENTO COVID-19 ARANCELES GRADO - DESCUENT...,20.0,90.26,MINISTERIO DEL INTERIOR - POLICIA NACIONAL DEL...,BANCOS ON LINE,PAGO EN BANCO,NO DEFINIDO,NO DEFINIDO,24.0,MUJER,NO,NO DEFINIDO,0,0,DSOF_3046,INGENIERIA DE REQUISITOS,0,APROBADA,7.54,26.39,10.0,35.0,8.77,26.31,7.46,26.11,8.0,28.0,8.5,25.5,8.37,83.7,1.0


In [ ]:
integrated__data_case4.to_csv('integrated-data-case04.csv', index = False)